In [1]:
#importing required modules

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
ses=tf.Session()

In [2]:
#preprocessing data (image into numpy array)
#splitting it for training and testing
#here train labels are one hot encoded

In [7]:
os.chdir("C:/Users/karthi/Desktop/image_classifier")
train_images=os.listdir("train_images/")
test_images=os.listdir("test_images/")
train_images_data=[]
test_images_data=[]
train_images_label=[]
for img in train_images:
    data=Image.open("train_images/{}".format(img))
    data=np.array(data.getdata()).reshape([1,128,128,3]).astype("float32")
    train_images_data.append(data)
for img in test_images:
    data=Image.open("test_images/{}".format(img))
    data=np.array(data.getdata()).reshape([1,128,128,3]).astype("float32")
    test_images_data.append(data)
for i in range(0,283):
    if i<139:
        train_images_label.append(np.array([1,0]).astype('float32'))
    else:
        train_images_label.append(np.array([0,1]).astype('float32'))

In [ ]:
#creating required weights and biases
#here number of epochs 20

In [8]:
x=tf.placeholder("float32",[1,128,128,3])
y=tf.placeholder('float32',[1,2])
conv_layer1={"weights":tf.Variable(tf.truncated_normal([3,3,3,3])),"bias":tf.Variable(tf.random_normal([3]))}
conv_layer2={"weights":tf.Variable(tf.truncated_normal([5,5,3,32])),"bias":tf.Variable(tf.random_normal([32]))}
conv_layer3={"weights":tf.Variable(tf.truncated_normal([6,6,32,64])),"bias":tf.Variable(tf.random_normal([64]))}
fc_layer1={"weights":tf.Variable(tf.truncated_normal([32*32*64,600])),"bias":tf.Variable(tf.random_normal([600]))}
out_layer={"weights":tf.Variable(tf.truncated_normal([600,2])),"bias":tf.Variable(tf.random_normal([2]))}
no_epoch=20

In [ ]:
#defining the convolution layer and fully connected layer 

In [9]:
def conv_classify(x):
    print("==========================================================")
    con1=tf.nn.conv2d(x,conv_layer1["weights"],strides=[1,1,1,1],padding="SAME")+conv_layer1["bias"]
    print("convolution layer1 {}   filter  3X3".format(con1.shape))
    con1=tf.nn.relu(con1)
    print("relu {}".format(con1.shape))
    print("==========================================================")
    con2=tf.nn.conv2d(con1,conv_layer2["weights"],strides=[1,1,1,1],padding="SAME")+conv_layer2["bias"]
    print("convolution layer2 {}   filter  5X5".format(con2.shape))
    con2=tf.nn.relu(con2)
    print("relu {}".format(con2.shape))
    con2=tf.nn.max_pool(con2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    print("maxpool layer {}   filter  2X2".format(con2.shape))
    print("==========================================================")
    con3=tf.nn.conv2d(con2,conv_layer3["weights"],strides=[1,1,1,1],padding="SAME")+conv_layer3["bias"]
    print("convolution layer3 {}   filter  6X6".format(con3.shape))
    con3=tf.nn.relu(con3)
    print("relu {}".format(con3.shape))
    con3=tf.nn.max_pool(con3,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    print("maxpool layer {}   filter  2X2".format(con3.shape))
    print("==========================================================")
    con3=tf.reshape(con3,[-1,32*32*64])
    fc=tf.add(tf.matmul(con3,fc_layer1["weights"]),fc_layer1["bias"])
    fc=tf.nn.sigmoid(fc)
    print("fully connected layer {}".format(fc.shape))
    fc=tf.nn.dropout(fc,0.75)
    out=tf.add(tf.matmul(fc,out_layer["weights"]),out_layer["bias"])
    out=tf.nn.sigmoid(out)
    print("output layer {}".format(out.shape))
    print("==========================================================")
    return out

In [ ]:
#training the  network 
#here it trains to find whether given image is cat or car

In [10]:
pred=conv_classify(x)
print("Training network")
cost=tf.reduce_mean(tf.square(pred-y))
optimiser=tf.train.AdamOptimizer().minimize(cost)
ses.run(tf.global_variables_initializer())
for epoch in range(0,no_epoch):
    epoch_loss=0
    for pic in range(0,283):
        _,cost_,y_,predi=ses.run([optimiser,cost,y,pred],feed_dict={x:train_images_data[pic],y:train_images_label[pic].reshape([1,2])})
        epoch_loss=epoch_loss+cost_
    print("epoch {} of {} total loss: {}".format(epoch+1,no_epoch,epoch_loss))
print("training completed")

convolution layer1 (1, 128, 128, 3)   filter  3X3
relu (1, 128, 128, 3)
convolution layer2 (1, 128, 128, 32)   filter  5X5
relu (1, 128, 128, 32)
maxpool layer (1, 64, 64, 32)   filter  2X2
convolution layer3 (1, 64, 64, 64)   filter  6X6
relu (1, 64, 64, 64)
maxpool layer (1, 32, 32, 64)   filter  2X2
fully connected layer (1, 600)
output layer (1, 2)
Training network
epoch 1 of 20 total loss: 139.8433881591624
epoch 2 of 20 total loss: 114.54857212211084
epoch 3 of 20 total loss: 109.76856463274288
epoch 4 of 20 total loss: 96.76864407090589
epoch 5 of 20 total loss: 72.9628610807757
epoch 6 of 20 total loss: 79.60532039504521
epoch 7 of 20 total loss: 49.66609136263518
epoch 8 of 20 total loss: 52.15510767518204
epoch 9 of 20 total loss: 39.490946154916934
epoch 10 of 20 total loss: 37.03677648686858
epoch 11 of 20 total loss: 30.89101963000808
epoch 12 of 20 total loss: 34.63225539487312
epoch 13 of 20 total loss: 23.594365963083206
epoch 14 of 20 total loss: 26.93971385915988
epoc

In [ ]:
#testing the images whether it predicting correctly or not

In [11]:
print("test images")
for i in range(0,10):
    ggg=ses.run(pred,feed_dict={x:test_images_data[i]})
    print(ggg)
    maxi=max(ggg[0])
    if ggg[0][0]==maxi:
        print("{} is cat".format(test_images[i])) 
    else:
        print("{} is car".format(test_images[i]))

test images
[[  6.42847226e-06   1.00000000e+00]]
1.png is car
[[  9.99999642e-01   1.84064266e-08]]
2.png is cat
[[  1.00000000e+00   7.16541919e-08]]
gvhvkvkj.png is cat
[[  1.00000000e+00   1.55696025e-08]]
obj14__175.png is cat
[[  9.99999285e-01   3.05524921e-08]]
obj14__245.png is cat
[[ 1.          0.00963312]]
obj14__70.png is cat
[[  2.68070662e-22   1.00000000e+00]]
obj6__230.png is car
[[  2.09741495e-11   1.00000000e+00]]
obj6__260.png is car
[[  4.58630436e-16   1.00000000e+00]]
obj6__355.png is car
[[  1.36555467e-13   1.00000000e+00]]
z.png is car
